# Eseguire il training di un modello di classificazione con Machine Learning automatizzato

Esistono molti tipi di algoritmo di Machine Learning che è possibile usare per eseguire il training di un modello e talvolta non è facile determinare l'algoritmo più efficace per i dati e i requisiti di stima specifici. Inoltre, è possibile influire significativamente sulle prestazioni predittive di un modello pre-elaborando i dati di training, usando tecniche come la normalizzazione, l'imputazione di funzionalità mancanti e altri. Nella ricerca di trovare il modello migliore per i requisiti, potrebbe essere necessario provare molte combinazioni di algoritmi e trasformazioni di pre-elaborazione; che richiede molto tempo e risorse di calcolo.

Azure Machine Learning consente di automatizzare il confronto dei modelli sottoposti a training usando algoritmi diversi e opzioni di pre-elaborazione. È possibile usare l'interfaccia visiva in [Azure Machine Learning Studio](https://ml/azure.com) o Python SDK (v2) per sfruttare questa funzionalità. Python SDK offre un maggiore controllo sulle impostazioni per il processo di Machine Learning automatizzato, ma l'interfaccia visiva è più semplice da usare.

## Prima di iniziare

È necessaria la versione più recente del pacchetto  **azureml-ai-ml** per eseguire il codice in questo notebook. Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguirlo `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Con i pacchetti SDK necessari installati, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati. È necessario solo l'ID sottoscrizione per completare il comando.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro nella parte superiore destra di Studio. Un riquadro verrà aperto a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Preparazione dei dati

Non è necessario creare uno script di training per Machine Learning automatizzato, ma è necessario caricare i dati di training. 

In questo caso si userà un set di dati contenente i dettagli dei pazienti del diabete. 

Per passare un set di dati come input a un processo di Machine Learning automatizzato, i dati devono essere in formato tabulare e includere una colonna di destinazione. Per l'interpretazione dei dati come set di dati tabulari, il set di dati di input deve essere **mlTable**.

Un asset dati MLTable è già stato creato durante la configurazione. È possibile esplorare l'asset dei dati passando alla pagina **Dati** . L'asset dati verrà recuperato qui specificando il nome `diabetes-training-table` e la versione `1`. 

In [ ]:
## Configurare il processo di Machine Learning automatizzato

A questo momento, è possibile configurare l'esperimento di Machine Learning automatizzato.

Quando si esegue il codice seguente, creerà un processo di Machine Learning automatizzato che:

- Usa il cluster di calcolo denominato `aml-cluster`
- Imposta `Diabetic` come colonna di destinazione
- Imposta `accuracy` come metrica primaria
- Timeout dopo `60` minuti di tempo totale di training 
- Esegue il training di un massimo di `5` modelli
- Nessun modello verrà sottoposto a training con l'algoritmo `LogisticRegression`

In [ ]:
## Eseguire un processo di Machine Learning automatizzato

OK, sei pronto per andare. Eseguire l'esperimento di Machine Learning automatizzato.

> Nota: questo potrebbe richiedere **qualche** tempo!

Durante l'esecuzione del processo, è possibile monitorarlo in Studio.

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:diabetes-training:1")

## Configure automated machine learning job

Now, you're ready to configure the automated machine learning experiment.

When you run the code below, it will create an automated machine learning job that:

- Uses the compute cluster named `aml-cluster`
- Sets `Diabetic` as the target column
- Sets `accuracy` as the primary metric
- Times out after `60` minutes of total training time 
- Trains a maximum of `5` models
- No model will be trained with the `LogisticRegression` algorithm

In [ ]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column_name="Diabetic",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

# set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"], 
    enable_onnx_compatible_models=True
)

## Run an automated machine learning job

OK, you're ready to go. Let's run the automated machine learning experiment.

> **Note**: This may take some time!

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

While the job is running, you can monitor it in the Studio.